In [3]:
import pandas as pd
import os 

os.chdir('..')
articles = pd.read_csv('Data/scored_articles.csv')

In [12]:
articles["date"] = pd.to_datetime(dict(
    year=articles["annee"],
    month=articles["mois"],
    day=articles["jour"]
))

articles["post_metoo"] = articles["date"] >= "2017-10-01"


In [13]:
articles

,Unnamed: 0.1,Unnamed: 0,identifiant,journal_clean,titre,annee,mois,jour,texte,keywords,...,score_final,voc_violence_final,hierar_final,portrait_victime_final,portrait_auteur_final,relation_final,meanisme_violence_final,stat_final,date,post_metoo
0,0,0,3a4723d9d754ca30c68ba8e420cd6683548f2af7ff7af3...,Libération,Viol collectif dans la nuit de la Saint-Sylves...,1995,1,6,quatre jeunes de 19 à 23 ans ont été écroués m...,"nuit, jeunes, collectif",...,0.714286,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1995-01-06,False
1,1,1,2b6ce23c0fbfd6e213b7cd196a4b12eda4323e322f5f35...,Libération,"En Moselle, une jeune femme tuée par son mari",1995,1,16,"une femme de 23 ans, mère d'une fillette de 2 ...","mari, ans",...,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,1995-01-16,False
2,2,5,2293f5c041bec1dcdfe52bbe2655e0bec9a449310f534b...,Libération,Narbonne: une femme tuée après une fusillade. ...,1995,4,11,"narbonne, correspondance au départ, un banal...","femme, heure, gendarmes, narbonne",...,0.285714,0.0,1.0,1.0,1.0,0.0,0.0,-1.0,1995-04-11,False
3,3,6,2e9c275968c7996e9407977dfb6aaacd7141ac5e7ed0a2...,Libération,Une femme enceinte agressée par des vigiles au...,1995,4,17,"des vigiles, appelés par la société de nettoya...","gare, vigiles, salariée, pei, femme",...,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,1995-04-17,False
4,4,12,b1987151ea7e9e657b7207398a434df106c3a9d8d3c709...,Libération,Une femme blesse son ex-ami avant de se suicider,1995,9,18,"le cadavre d'un femme, tuée par arme à feu, a ...",arme,...,0.000000,0.0,1.0,1.0,0.0,0.0,-1.0,-1.0,1995-09-18,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3091,3091,12150,cdd9ae89b1a8738a4e2eca205b6918093b627b7e8275d1...,Le Figaro,Près de Lyon : une jeune femme pompier agressé...,2024,12,2,une jeune femme pompier a été agressée sexuell...,"jeune, intervention, lyon, d’une, pompier, agr...",...,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2024-12-02,True
3092,3092,12154,377988b4b18f66bdf316259434127bc49e39ac016edacd...,Le Figaro,Italie: un homme condamné à la prison à vie po...,2024,12,3,la cour d'assises de venise a condamné mardi 3...,"relancé, pays, venise, cour, débat, féminicide",...,0.571429,1.0,1.0,1.0,1.0,0.0,1.0,-1.0,2024-12-03,True
3093,3093,12155,1a7689437dc0d5a11e37df86ddd7d9ea4c37195a7ecdbc...,Le Point,Harvey Weinstein est de nouveau hospitalisé,2024,12,4,harvey weinstein est de retour à l'hôpital. l'...,"producteur, new, harvey, york, hospitalisé, hô...",...,0.714286,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,2024-12-04,True
3094,3094,12159,20d7c133c259fe5e149d1cddfa8be63aa616233362596f...,Le Figaro,«Adèle Haenel et ses 12 ans étaient d’une sens...,2024,12,4,au dernier étage d’un immeuble de la rue jean-...,"violences, christophe, adèle, réalisateur, d’u...",...,0.714286,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,2024-12-04,True


In [14]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
label_cols = [ "score",
    "voc_violence", "hierar", "portrait_victime",
    "portrait_auteur", "relation", "meanisme_violence", "stat"
]

# 1. Création des groupes avant/après MeToo
pre = articles[articles["post_metoo"] == False]
post = articles[articles["post_metoo"] == True]


vars_to_compare = [col + "_final" for col in label_cols]

results = []

for var in vars_to_compare:
    pre_vals = pre[var].dropna()
    post_vals = post[var].dropna()
    
    mean_pre = pre_vals.mean()
    mean_post = post_vals.mean()
    diff = mean_post - mean_pre
    
    tstat, pval = ttest_ind(post_vals, pre_vals, equal_var=False)
    
    results.append({
        "variable": var,
        "mean_pre": round(mean_pre, 3),
        "mean_post": round(mean_post, 3),
        "diff": round(diff, 3),
        "p_value": round(pval, 4)
    })

df_results = pd.DataFrame(results)


In [15]:
df_results

,variable,mean_pre,mean_post,diff,p_value
0,score_final,0.122,0.604,0.482,0.0
1,voc_violence_final,0.284,0.785,0.501,0.0
2,hierar_final,0.519,0.736,0.217,0.0
3,portrait_victime_final,0.403,0.814,0.410,0.0
4,portrait_auteur_final,0.364,0.801,0.437,0.0
5,relation_final,0.216,0.674,0.459,0.0
6,meanisme_violence_final,-0.054,0.644,0.698,0.0
7,stat_final,-0.878,-0.227,0.651,0.0
